In [1]:
from os.path import dirname, join, isfile, isdir
from datetime import datetime, timedelta
from unidecode import unidecode
from hashlib import sha256
from os import makedirs
from sys import exit
import pandas as pd
import codecs


from bulletin import __file__ as __root__
from bulletin.data.casos_confirmados import CasosConfirmados
from bulletin.data.notifica import Notifica
from bulletin.commom import static
from bulletin.commom.normalize import normalize_text, normalize_labels, normalize_number, normalize_municipios, normalize_igbe, normalize_hash, data_hash

In [2]:
curitiba = pd.read_excel('input/curitiba.xlsx', 
                         usecols='B:E,J',
                         converters = {
                               'classificacao': normalize_text,
                               'nome': normalize_text,
                               'idade': lambda x: normalize_number(x,fill=0),
                               'mun_resid': normalize_text
                        })

curitiba.groupby('classificacao')[['nome']].count()

,nome
classificacao,
CONFIRMADO,133594
CONFIRMADO (1a),260
CONFIRMADO (2A ?),238
CONFIRMADO (2a?),265
CONFIRMADO(1a?),2
CONFIRMADO(2a OU COMPLIC?),2
CONFIRMADO(2a OU SEQUELA?),1
CONFIRMADO?,66
DESCARTADO,4171


In [3]:
curitiba = curitiba.loc[curitiba['classificacao']=='CONFIRMADO'].copy()

In [4]:
curitiba['hash'] = curitiba.apply(lambda row: normalize_hash(row['nome'])+str(row['idade'])+normalize_hash(row['mun_resid']), axis=1)
curitiba['hash_less'] = curitiba.apply(lambda row: normalize_hash(row['nome'])+str(row['idade']-1)+normalize_hash(row['mun_resid']), axis=1)
curitiba['hash_more'] = curitiba.apply(lambda row: normalize_hash(row['nome'])+str(row['idade']+1)+normalize_hash(row['mun_resid']), axis=1)

In [5]:
cc = CasosConfirmados()
# cc.update()
casos = cc.get_casos()

In [6]:
nos_casos = curitiba.loc[curitiba['hash'].isin(casos['hash'])].index.tolist()
nos_casos += curitiba.loc[curitiba['hash'].isin(casos['hash_less'])].index.tolist()
nos_casos += curitiba.loc[curitiba['hash'].isin(casos['hash_more'])].index.tolist()

In [7]:
nao_cc = set(curitiba.index.tolist()) - set(nos_casos)
curitiba.loc[nao_cc].sort_index()
len(nao_cc)

63029

In [8]:
notifica = Notifica()
notifica.load()
nt_casos = notifica.get_casos()

In [9]:
no_notifica = curitiba.loc[curitiba['hash'].isin(nt_casos['hash_idade_resid'])].index.tolist()
no_notifica += curitiba.loc[curitiba['hash_less'].isin(nt_casos['hash_idade_resid'])].index.tolist()
no_notifica += curitiba.loc[curitiba['hash_more'].isin(nt_casos['hash_idade_resid'])].index.tolist()

In [10]:
nao_nt = set(curitiba.index.tolist()) - set(no_notifica)
curitiba.loc[nao_nt].sort_index()
len(nao_nt)

61635

In [11]:
len(nao_cc) - len(nao_nt)

1394